## Identifying and resolving conflicts in a batch of VI files

ELGs - part 6 is sets 26 to 30

There are 749 visual inspections of a total of 250 unique objects
That gives 2.996 inspections per object.

49 conflicts

In [2]:
save_filename='truth_table_ELGs_sets26_30.txt'

In [3]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

#import desispec
# library location will change ..
sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [4]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
del obs_db['67230-20200314']
obs_db.keys()

dict_keys(['67230-20200315'])

In [5]:
pd.set_option('display.max_rows', 20)

In [6]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

In [7]:
vi_files = ['desi-vi_SV0_ELG_tile67230_night20200315_26_HSH.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_26_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_26_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_27_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_27_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_27_BJW.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_28_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_28_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_28_BJW.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_29_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_29_DRP.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_29_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_30_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_30_SEM.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_30_ACE.csv']

In [8]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = " , ", engine='python')

for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = " , ", engine='python')
    vi = vi.append(vi2, ignore_index=True)
    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TargetID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_SV0_ELG_tile67230_night20200315_26_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_26_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_27_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_27_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_27_BJW.csv
desi-vi_SV0_ELG_tile67230_night20200315_28_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_28_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_28_BJW.csv
desi-vi_SV0_ELG_tile67230_night20200315_29_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_29_DRP.csv
desi-vi_SV0_ELG_tile67230_night20200315_29_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_30_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_30_SEM.csv
desi-vi_SV0_ELG_tile67230_night20200315_30_ACE.csv
There are 749 visual inspections of a total of 250 unique objects
That gives 2.996 inspections per object.


In [9]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment'],
      dtype='object')

### Merge with zbest files
Add: fiberID, delta_chi2, flux information,.. anything else?


In [10]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

vi = vi.merge(tf_df, how='left', on='TargetID')
vi = vi.merge(tspec_df, how='left', on='TargetID')

In [11]:
print(vi.keys())
print(len(vi))

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')
749


### Adding a bunch of useful columns

In [12]:
vi = VI.prep_new_columns(vi)

In [13]:
#check all the new columns (keys) have been added correctly
display(vi)
print(vi.keys())

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35185772160090388,-1,0.33.0.dev3890,0,GALAXY,0.8145,HSH,0,R,--,...,7.868769,0.8145,GALAXY,0.666667,2,0.094655,0.8145,--|Low S/N; Model fits sky residuals|Based on ...,3,none
1,35185772160091169,-1,0.33.0.dev3890,0,GALAXY,1.1189,HSH,4,--,--,...,97.994071,1.1189,GALAXY,4.000000,0,0.000142,1.1190,--| Broad [OII] emission|--,3,none
2,35185772164285442,-1,0.33.0.dev3890,0,GALAXY,1.1216,HSH,3,--,--,...,75.235923,1.1216,GALAXY,3.666667,1,0.000000,1.1216,--|--|--,3,none
3,35185772160090339,-1,0.33.0.dev3890,0,GALAXY,1.6864,HSH,0,R,--,...,0.974392,1.6864,GALAXY,1.000000,3,0.354510,1.6864,--| Some blue continuum; no features|Based on OII,3,none
4,35185772160091340,-1,0.33.0.dev3890,0,GALAXY,1.0748,HSH,4,--,--,...,497.939740,1.0748,GALAXY,4.000000,0,0.000000,1.0748,--|--|--,3,none
5,35185772160091680,-1,0.33.0.dev3890,0,GALAXY,1.3426,HSH,3,--,--,...,140.420065,1.3426,GALAXY,3.666667,1,0.000000,1.3426,--|--|--,3,none
6,35185772160092167,-1,0.33.0.dev3890,0,GALAXY,0.0941,HSH,4,R,0.84,...,779.433202,0.8400,GALAXY,4.000000,0,0.686957,0.0941,--|Model fits sky residuals|--,3,none
7,35185772151701749,-1,0.33.0.dev3890,0,GALAXY,1.6163,HSH,0,RS,--,...,56.813565,1.6163,GALAXY,2.000000,3,0.000000,1.6163,--|Calibration features around 5800 A; possibl...,3,none
8,35185772151701829,-1,0.33.0.dev3890,0,GALAXY,0.4462,HSH,0,RS,--,...,66.079907,0.4462,GALAXY,0.666667,2,0.446200,0.4462,--| Calibration features around 5800A| Poor co...,3,none
9,35185772151702242,-1,0.33.0.dev3890,0,GALAXY,0.9726,HSH,3,S,--,...,61.165908,0.9726,GALAXY,3.666667,1,0.000000,0.9726,--| Calibration features around 6000 A| Poor c...,3,none


Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [14]:
vi_gp = vi.groupby(['TargetID'])
vi_conflict = VI.find_conflicts(vi_gp)

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [15]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185772147512575 35185772151701749 35185772151703841 35185772151706089
 35185772151706762 35185772155896164 35185772155896438 35185772155896628
 35185772155897823 35185772155900426 35185772155901185 35185772160090339
 35185772160092167 35185772160094488 35185772164284927 35185772164286546
 35185772164286597 35185772164288083 35185772164288175 35185772164289259
 35185772168481893 35185772168481920 35185772172673676 35185772172673848
 35185772172674321 35185772172676812 35185772172677547 35185772176869420
 35185772176871722 35185772181062339 35185772181063930 35185772181065428
 35185772181065499 35185772185256183 35185772185256282 35185772185256510
 35185772185257394 35185772185257398 35185772185257520 35185772185259110
 35185772185259307 35185772189451229 35185778174722907 35185778174727087
 35185778174727095 35185778178918499 35185778183111392 35185778183115498
 35185778187305121]
Total number of conflicts to resolve:  49


In [16]:
unique_targets[1]

35185772151701749

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [17]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TargetID==unique_targets[conflict_id]][['TargetID', 
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    # VI interface in notebook
    if show_spectra:
        spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [18]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [19]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
23,35185772147512575,GALAXY,1.6979,HSH,2,R,--,--,--,1.6979,GALAXY,2.0,2,1.6979,1.6979,--| Some continuum; no features|--,3,none
73,35185772147512575,GALAXY,1.6979,TWL,1,--,--,--,Some continuum; no features,1.6979,GALAXY,2.0,2,1.6979,1.6979,--| Some continuum; no features|--,3,none
123,35185772147512575,GALAXY,1.6979,ACE,3,--,0.0000,STAR,--,0.0000,STAR,2.0,2,1.6979,1.6979,--| Some continuum; no features|--,3,none


In [20]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6979
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and spectype. low SNR, hardly a continuum. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
23,35185772147512575,-1,0.33.0.dev3890,0,GALAXY,1.6979,HSH,1,R,--,...,0.35824,1.6979,GALAXY,2.0,2,1.6979,1.6979,--| Some continuum; no features|--,3,"disagreement in flag and spectype. low SNR, ha..."
73,35185772147512575,-1,0.33.0.dev3890,0,GALAXY,1.6979,TWL,1,--,--,...,0.35824,1.6979,GALAXY,2.0,2,1.6979,1.6979,--| Some continuum; no features|--,3,"disagreement in flag and spectype. low SNR, ha..."
123,35185772147512575,-1,0.33.0.dev3890,0,GALAXY,1.6979,ACE,1,--,0.0000,...,0.35824,1.6979,GALAXY,2.0,2,1.6979,1.6979,--| Some continuum; no features|--,3,"disagreement in flag and spectype. low SNR, ha..."


conflict ID =  0


In [21]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
7,35185772151701749,GALAXY,1.6163,HSH,0,RS,--,--,--,1.6163,GALAXY,2.0,3,1.6163,0.0,--|Calibration features around 5800 A; possibl...,3,none
57,35185772151701749,GALAXY,1.6163,TWL,3,--,--,--,Calibration features around 5800 A; possible [...,1.6163,GALAXY,2.0,3,1.6163,0.0,--|Calibration features around 5800 A; possibl...,3,none
107,35185772151701749,GALAXY,1.6163,ACE,3,--,--,--,--,1.6163,GALAXY,2.0,3,1.6163,0.0,--|Calibration features around 5800 A; possibl...,3,none


In [22]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6163
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but just about resolved OII. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
7,35185772151701749,-1,0.33.0.dev3890,0,GALAXY,1.6163,HSH,2.5,RS,--,...,56.813565,1.6163,GALAXY,2.0,3,0.0,1.6163,--|Calibration features around 5800 A; possibl...,3,disagreement in flag. broadened but just about...
57,35185772151701749,-1,0.33.0.dev3890,0,GALAXY,1.6163,TWL,2.5,--,--,...,56.813565,1.6163,GALAXY,2.0,3,0.0,1.6163,--|Calibration features around 5800 A; possibl...,3,disagreement in flag. broadened but just about...
107,35185772151701749,-1,0.33.0.dev3890,0,GALAXY,1.6163,ACE,2.5,--,--,...,56.813565,1.6163,GALAXY,2.0,3,0.0,1.6163,--|Calibration features around 5800 A; possibl...,3,disagreement in flag. broadened but just about...


conflict ID =  1


In [23]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
13,35185772151703841,GALAXY,1.2142,HSH,2.0,S,--,--,--,1.2142,GALAXY,3.333333,2,1.2142,0.0,--| Broad [OII] ; calibration features around ...,3,none
63,35185772151703841,GALAXY,1.2142,TWL,4.0,S,--,--,Broad [OII] ; calibration features around 5800 A,1.2142,GALAXY,3.333333,2,1.2142,0.0,--| Broad [OII] ; calibration features around ...,3,none
113,35185772151703841,GALAXY,1.2142,ACE,4.0,--,--,--,Poor continuum subtraction at 5800-6800A.,1.2142,GALAXY,3.333333,2,1.2142,0.0,--| Broad [OII] ; calibration features around ...,3,none


In [24]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but OII secures z. calibration issues. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
13,35185772151703841,-1,0.33.0.dev3890,0,GALAXY,1.2142,HSH,3.0,S,--,...,71.271587,1.2142,GALAXY,3.333333,2,0.0,1.2142,--| Broad [OII] ; calibration features around ...,3,"disagreement in flag, but OII secures z. calib..."
63,35185772151703841,-1,0.33.0.dev3890,0,GALAXY,1.2142,TWL,3.0,S,--,...,71.271587,1.2142,GALAXY,3.333333,2,0.0,1.2142,--| Broad [OII] ; calibration features around ...,3,"disagreement in flag, but OII secures z. calib..."
113,35185772151703841,-1,0.33.0.dev3890,0,GALAXY,1.2142,ACE,3.0,--,--,...,71.271587,1.2142,GALAXY,3.333333,2,0.0,1.2142,--| Broad [OII] ; calibration features around ...,3,"disagreement in flag, but OII secures z. calib..."


conflict ID =  2


In [25]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
25,35185772151706089,QSO,5.1629,HSH,0.0,R,--,--,--,5.1629,QSO,1.0,3,5.1629,5.1629,--|--|--,3,none
75,35185772151706089,QSO,5.1629,TWL,0.0,--,--,--,--,5.1629,QSO,1.0,3,5.1629,5.1629,--|--|--,3,none
125,35185772151706089,QSO,5.1629,ACE,3.0,--,0.0000,STAR,--,0.0000,STAR,1.0,3,5.1629,5.1629,--|--|--,3,none


In [26]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 5.1629
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and spectype. too low SNR to tell, not a QSO. calibration issues. take RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
25,35185772151706089,-1,0.33.0.dev3890,0,QSO,5.1629,HSH,1.0,R,--,...,4.326146,5.1629,QSO,1.0,3,5.1629,5.1629,--|--|--,3,disagreement in flag and spectype. too low SNR...
75,35185772151706089,-1,0.33.0.dev3890,0,QSO,5.1629,TWL,1.0,--,--,...,4.326146,5.1629,QSO,1.0,3,5.1629,5.1629,--|--|--,3,disagreement in flag and spectype. too low SNR...
125,35185772151706089,-1,0.33.0.dev3890,0,QSO,5.1629,ACE,1.0,--,0.0000,...,4.326146,5.1629,QSO,1.0,3,5.1629,5.1629,--|--|--,3,disagreement in flag and spectype. too low SNR...


conflict ID =  3


In [27]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
28,35185772151706762,GALAXY,1.4698,HSH,2.0,--,--,--,--,1.4698,GALAXY,3.0,2,1.4698,0.0,--|--|--,3,none
78,35185772151706762,GALAXY,1.4698,TWL,4.0,--,--,--,--,1.4698,GALAXY,3.0,2,1.4698,0.0,--|--|--,3,none
128,35185772151706762,GALAXY,1.4698,ACE,3.0,--,--,--,--,1.4698,GALAXY,3.0,2,1.4698,0.0,--|--|--,3,none


In [28]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, z secured by doublet. take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
28,35185772151706762,-1,0.33.0.dev3890,0,GALAXY,1.4698,HSH,3.0,--,--,...,80.773574,1.4698,GALAXY,3.0,2,0.0,1.4698,--|--|--,3,"flag disagreement, z secured by doublet. take ..."
78,35185772151706762,-1,0.33.0.dev3890,0,GALAXY,1.4698,TWL,3.0,--,--,...,80.773574,1.4698,GALAXY,3.0,2,0.0,1.4698,--|--|--,3,"flag disagreement, z secured by doublet. take ..."
128,35185772151706762,-1,0.33.0.dev3890,0,GALAXY,1.4698,ACE,3.0,--,--,...,80.773574,1.4698,GALAXY,3.0,2,0.0,1.4698,--|--|--,3,"flag disagreement, z secured by doublet. take ..."


conflict ID =  4


In [29]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
14,35185772155896164,GALAXY,1.2007,HSH,2.0,S,--,--,--,1.2007,GALAXY,3.333333,2,1.2007,0.0,--|Calibration features around 5800 A| Poor c...,3,none
64,35185772155896164,GALAXY,1.2007,TWL,4.0,S,--,--,Calibration features around 5800 A,1.2007,GALAXY,3.333333,2,1.2007,0.0,--|Calibration features around 5800 A| Poor c...,3,none
114,35185772155896164,GALAXY,1.2007,ACE,4.0,--,--,--,Poor continuum subtraction at 5800-6800A.,1.2007,GALAXY,3.333333,2,1.2007,0.0,--|Calibration features around 5800 A| Poor c...,3,none


In [30]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, doublet secures z. take mean. poor sky subtraction. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
14,35185772155896164,-1,0.33.0.dev3890,0,GALAXY,1.2007,HSH,4.0,S,--,...,71.714313,1.2007,GALAXY,3.333333,2,0.0,1.2007,--|Calibration features around 5800 A| Poor c...,3,"flag disagreement, doublet secures z. take mea..."
64,35185772155896164,-1,0.33.0.dev3890,0,GALAXY,1.2007,TWL,4.0,S,--,...,71.714313,1.2007,GALAXY,3.333333,2,0.0,1.2007,--|Calibration features around 5800 A| Poor c...,3,"flag disagreement, doublet secures z. take mea..."
114,35185772155896164,-1,0.33.0.dev3890,0,GALAXY,1.2007,ACE,4.0,--,--,...,71.714313,1.2007,GALAXY,3.333333,2,0.0,1.2007,--|Calibration features around 5800 A| Poor c...,3,"flag disagreement, doublet secures z. take mea..."


conflict ID =  5


In [31]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
16,35185772155896438,GALAXY,0.4446,HSH,2.0,RS,--,--,--,0.4446,GALAXY,3.0,2,0.4446,0.438807,--| Possible [OII] at z~1.0785; bad calibratio...,3,none
66,35185772155896438,GALAXY,0.4446,TWL,3.0,RS,1.0785,--,Possible [OII] at z~1.0785; bad calibration,1.0785,GALAXY,3.0,2,0.4446,0.438807,--| Possible [OII] at z~1.0785; bad calibratio...,3,none
116,35185772155896438,GALAXY,0.4446,ACE,4.0,R,1.0783,GALAXY,Poor continuum subtraction at 5800-6800A.,1.0783,GALAXY,3.0,2,0.4446,0.438807,--| Possible [OII] at z~1.0785; bad calibratio...,3,none


In [32]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0784
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/3 inspectors give same VI z, based on OII. take with flag 3. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
16,35185772155896438,-1,0.33.0.dev3890,0,GALAXY,0.4446,HSH,3.0,RS,--,...,70.528819,1.0784,GALAXY,3.0,2,0.438807,0.4446,--| Possible [OII] at z~1.0785; bad calibratio...,3,disagreement in flag and z. 2/3 inspectors giv...
66,35185772155896438,-1,0.33.0.dev3890,0,GALAXY,0.4446,TWL,3.0,RS,1.0785,...,70.528819,1.0784,GALAXY,3.0,2,0.438807,0.4446,--| Possible [OII] at z~1.0785; bad calibratio...,3,disagreement in flag and z. 2/3 inspectors giv...
116,35185772155896438,-1,0.33.0.dev3890,0,GALAXY,0.4446,ACE,3.0,R,1.0783,...,70.528819,1.0784,GALAXY,3.0,2,0.438807,0.4446,--| Possible [OII] at z~1.0785; bad calibratio...,3,disagreement in flag and z. 2/3 inspectors giv...


conflict ID =  6


In [33]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
17,35185772155896628,GALAXY,0.4439,HSH,0.0,RS,--,--,--,0.4439,GALAXY,1.0,3,0.4439,0.258397,--|Bad calibration around 5800 A| Poor continu...,3,none
67,35185772155896628,GALAXY,0.4439,TWL,0.0,S,--,--,Bad calibration around 5800 A,0.4439,GALAXY,1.0,3,0.4439,0.258397,--|Bad calibration around 5800 A| Poor continu...,3,none
117,35185772155896628,GALAXY,0.4439,ACE,3.0,--,0.8170,GALAXY,Poor continuum subtraction at 5800-6800A.,0.8170,GALAXY,1.0,3,0.4439,0.258397,--|Bad calibration around 5800 A| Poor continu...,3,none


In [34]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4439
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 0
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no signal, calibration issues. flag 0.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
17,35185772155896628,-1,0.33.0.dev3890,0,GALAXY,0.4439,HSH,0.0,RS,--,...,9.069301,0.4439,GALAXY,1.0,3,0.258397,0.4439,--|Bad calibration around 5800 A| Poor continu...,3,"disagreement in flag and z. no signal, calibra..."
67,35185772155896628,-1,0.33.0.dev3890,0,GALAXY,0.4439,TWL,0.0,S,--,...,9.069301,0.4439,GALAXY,1.0,3,0.258397,0.4439,--|Bad calibration around 5800 A| Poor continu...,3,"disagreement in flag and z. no signal, calibra..."
117,35185772155896628,-1,0.33.0.dev3890,0,GALAXY,0.4439,ACE,0.0,--,0.8170,...,9.069301,0.4439,GALAXY,1.0,3,0.258397,0.4439,--|Bad calibration around 5800 A| Poor continu...,3,"disagreement in flag and z. no signal, calibra..."


conflict ID =  7


In [35]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
20,35185772155897823,GALAXY,0.0528,HSH,4.0,S,--,--,--,0.0528,GALAXY,2.333333,5,0.0528,0.0,--| Calibration features around 5800 A| Poor ...,3,none
70,35185772155897823,GALAXY,0.0528,TWL,4.0,S,--,--,Calibration features around 5800 A,0.0528,GALAXY,2.333333,5,0.0528,0.0,--| Calibration features around 5800 A| Poor ...,3,none
120,35185772155897823,GALAXY,0.0528,ACE,-1.0,--,--,--,Poor continuum subtraction at 5800-6800A.,0.0528,GALAXY,2.333333,5,0.0528,0.0,--| Calibration features around 5800 A| Poor ...,3,none


In [36]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'one inspector forgot to set a flag. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
20,35185772155897823,-1,0.33.0.dev3890,0,GALAXY,0.0528,HSH,4.0,S,--,...,5131.73613,0.0528,GALAXY,2.333333,5,0.0,0.0528,--| Calibration features around 5800 A| Poor ...,3,one inspector forgot to set a flag. z is secure.
70,35185772155897823,-1,0.33.0.dev3890,0,GALAXY,0.0528,TWL,4.0,S,--,...,5131.73613,0.0528,GALAXY,2.333333,5,0.0,0.0528,--| Calibration features around 5800 A| Poor ...,3,one inspector forgot to set a flag. z is secure.
120,35185772155897823,-1,0.33.0.dev3890,0,GALAXY,0.0528,ACE,4.0,--,--,...,5131.73613,0.0528,GALAXY,2.333333,5,0.0,0.0528,--| Calibration features around 5800 A| Poor ...,3,one inspector forgot to set a flag. z is secure.


conflict ID =  8


In [37]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
30,35185772155900426,GALAXY,1.4637,HSH,2.0,S,--,--,--,1.4637,GALAXY,3.0,2,1.4637,0.0,--| Broad [OII] emission|--,3,none
80,35185772155900426,GALAXY,1.4637,TWL,3.0,--,--,--,Broad [OII] emission,1.4637,GALAXY,3.0,2,1.4637,0.0,--| Broad [OII] emission|--,3,none
130,35185772155900426,GALAXY,1.4637,ACE,4.0,--,--,--,--,1.4637,GALAXY,3.0,2,1.4637,0.0,--| Broad [OII] emission|--,3,none


In [38]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved OII line. very broad profile, accept z with flag 2.5. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
30,35185772155900426,-1,0.33.0.dev3890,0,GALAXY,1.4637,HSH,2.5,S,--,...,112.835294,1.4637,GALAXY,3.0,2,0.0,1.4637,--| Broad [OII] emission|--,3,disagreement in flag. broadened but unresolved...
80,35185772155900426,-1,0.33.0.dev3890,0,GALAXY,1.4637,TWL,2.5,--,--,...,112.835294,1.4637,GALAXY,3.0,2,0.0,1.4637,--| Broad [OII] emission|--,3,disagreement in flag. broadened but unresolved...
130,35185772155900426,-1,0.33.0.dev3890,0,GALAXY,1.4637,ACE,2.5,--,--,...,112.835294,1.4637,GALAXY,3.0,2,0.0,1.4637,--| Broad [OII] emission|--,3,disagreement in flag. broadened but unresolved...


conflict ID =  9


In [39]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=True)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
34,35185772155901185,GALAXY,1.4625,HSH,2.0,S,--,--,--,1.4625,GALAXY,3.0,2,1.4625,0.0,--| Triple-peak [OII] emission|--,3,none
84,35185772155901185,GALAXY,1.4625,TWL,4.0,--,--,--,Triple-peak [OII] emission,1.4625,GALAXY,3.0,2,1.4625,0.0,--| Triple-peak [OII] emission|--,3,none
134,35185772155901185,GALAXY,1.4625,ACE,3.0,--,--,--,--,1.4625,GALAXY,3.0,2,1.4625,0.0,--| Triple-peak [OII] emission|--,3,none


Loading BokehJS ...

In [40]:
c=3e5
print(2/9182 * c)

65.3452406883032


In [41]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. triple peaked possibly broadened line.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
34,35185772155901185,-1,0.33.0.dev3890,0,GALAXY,1.4625,HSH,3.0,S,--,...,315.156474,1.4625,GALAXY,3.0,2,0.0,1.4625,--| Triple-peak [OII] emission|--,3,"flag disagreement, take mean. triple peaked po..."
84,35185772155901185,-1,0.33.0.dev3890,0,GALAXY,1.4625,TWL,3.0,--,--,...,315.156474,1.4625,GALAXY,3.0,2,0.0,1.4625,--| Triple-peak [OII] emission|--,3,"flag disagreement, take mean. triple peaked po..."
134,35185772155901185,-1,0.33.0.dev3890,0,GALAXY,1.4625,ACE,3.0,--,--,...,315.156474,1.4625,GALAXY,3.0,2,0.0,1.4625,--| Triple-peak [OII] emission|--,3,"flag disagreement, take mean. triple peaked po..."


conflict ID =  10


In [42]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
3,35185772160090339,GALAXY,1.6864,HSH,0.0,R,--,--,--,1.6864,GALAXY,1.0,3,1.6864,0.35451,--| Some blue continuum; no features|Based on OII,3,none
53,35185772160090339,GALAXY,1.6864,TWL,0.0,--,--,--,Some blue continuum; no features,1.6864,GALAXY,1.0,3,1.6864,0.35451,--| Some blue continuum; no features|Based on OII,3,none
103,35185772160090339,GALAXY,1.6864,ACE,3.0,R,0.9833,GALAXY,Based on OII,0.9833,GALAXY,1.0,3,1.6864,0.35451,--| Some blue continuum; no features|Based on OII,3,none


In [43]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6864
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag and z disagreement. very poor signal, take RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
3,35185772160090339,-1,0.33.0.dev3890,0,GALAXY,1.6864,HSH,1.0,R,--,...,0.974392,1.6864,GALAXY,1.0,3,0.35451,1.6864,--| Some blue continuum; no features|Based on OII,3,"flag and z disagreement. very poor signal, tak..."
53,35185772160090339,-1,0.33.0.dev3890,0,GALAXY,1.6864,TWL,1.0,--,--,...,0.974392,1.6864,GALAXY,1.0,3,0.35451,1.6864,--| Some blue continuum; no features|Based on OII,3,"flag and z disagreement. very poor signal, tak..."
103,35185772160090339,-1,0.33.0.dev3890,0,GALAXY,1.6864,ACE,1.0,R,0.9833,...,0.974392,1.6864,GALAXY,1.0,3,0.35451,1.6864,--| Some blue continuum; no features|Based on OII,3,"flag and z disagreement. very poor signal, tak..."


conflict ID =  11


In [44]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
6,35185772160092167,GALAXY,0.0941,HSH,4.0,R,0.84,--,--,0.8400,GALAXY,4.0,0,0.0941,0.686957,--|Model fits sky residuals|--,3,none
56,35185772160092167,GALAXY,0.0941,TWL,4.0,RS,0.8457,--,Model fits sky residuals,0.8457,GALAXY,4.0,0,0.0941,0.686957,--|Model fits sky residuals|--,3,none
106,35185772160092167,GALAXY,0.0941,ACE,4.0,R,--,--,--,0.0941,GALAXY,4.0,0,0.0941,0.686957,--|Model fits sky residuals|--,3,none


In [45]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8457
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/3 inspectors suggest secure VI z based on doublet missed by rr.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
6,35185772160092167,-1,0.33.0.dev3890,0,GALAXY,0.0941,HSH,3.0,R,0.84,...,779.433202,0.8457,GALAXY,4.0,0,0.686957,0.0941,--|Model fits sky residuals|--,3,disagreement in flag and z. 2/3 inspectors sug...
56,35185772160092167,-1,0.33.0.dev3890,0,GALAXY,0.0941,TWL,3.0,RS,0.8457,...,779.433202,0.8457,GALAXY,4.0,0,0.686957,0.0941,--|Model fits sky residuals|--,3,disagreement in flag and z. 2/3 inspectors sug...
106,35185772160092167,-1,0.33.0.dev3890,0,GALAXY,0.0941,ACE,3.0,R,--,...,779.433202,0.8457,GALAXY,4.0,0,0.686957,0.0941,--|Model fits sky residuals|--,3,disagreement in flag and z. 2/3 inspectors sug...


conflict ID =  12


In [46]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
36,35185772160094488,GALAXY,1.4415,HSH,3.0,RC,--,QSO,--,1.4415,QSO,3.333333,1,1.4415,0.312431,--| Possibly QSOl point source|--,3,none
86,35185772160094488,GALAXY,1.4415,TWL,3.0,RC,2.1999,QSO,Possibly QSOl point source,2.1999,QSO,3.333333,1,1.4415,0.312431,--| Possibly QSOl point source|--,3,none
136,35185772160094488,GALAXY,1.4415,ACE,4.0,--,2.2043,QSO,--,2.2043,QSO,3.333333,1,1.4415,0.312431,--| Possibly QSOl point source|--,3,none


In [47]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 2.2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. QSO missed by rr.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
36,35185772160094488,-1,0.33.0.dev3890,0,GALAXY,1.4415,HSH,4.0,RC,--,...,4.44138,2.2,QSO,3.333333,1,0.312431,1.4415,--| Possibly QSOl point source|--,3,disagreement in flag. QSO missed by rr.
86,35185772160094488,-1,0.33.0.dev3890,0,GALAXY,1.4415,TWL,4.0,RC,2.1999,...,4.44138,2.2,QSO,3.333333,1,0.312431,1.4415,--| Possibly QSOl point source|--,3,disagreement in flag. QSO missed by rr.
136,35185772160094488,-1,0.33.0.dev3890,0,GALAXY,1.4415,ACE,4.0,--,2.2043,...,4.44138,2.2,QSO,3.333333,1,0.312431,1.4415,--| Possibly QSOl point source|--,3,disagreement in flag. QSO missed by rr.


conflict ID =  13


In [48]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
46,35185772164284927,GALAXY,1.2935,HSH,2.0,--,--,--,--,1.2935,GALAXY,3.0,2,1.2935,0.0,--|--|--,3,none
96,35185772164284927,GALAXY,1.2935,TWL,4.0,--,--,--,--,1.2935,GALAXY,3.0,2,1.2935,0.0,--|--|--,3,none
146,35185772164284927,GALAXY,1.2935,ACE,3.0,--,--,--,--,1.2935,GALAXY,3.0,2,1.2935,0.0,--|--|--,3,none


In [49]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, secure z. take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
46,35185772164284927,-1,0.33.0.dev3890,0,GALAXY,1.2935,HSH,3.0,--,--,...,72.347411,1.2935,GALAXY,3.0,2,0.0,1.2935,--|--|--,3,"flag disagreement, secure z. take mean."
96,35185772164284927,-1,0.33.0.dev3890,0,GALAXY,1.2935,TWL,3.0,--,--,...,72.347411,1.2935,GALAXY,3.0,2,0.0,1.2935,--|--|--,3,"flag disagreement, secure z. take mean."
146,35185772164284927,-1,0.33.0.dev3890,0,GALAXY,1.2935,ACE,3.0,--,--,...,72.347411,1.2935,GALAXY,3.0,2,0.0,1.2935,--|--|--,3,"flag disagreement, secure z. take mean."


conflict ID =  14


In [50]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
40,35185772164286546,GALAXY,1.508,HSH,2.0,--,--,--,--,1.508,GALAXY,2.666667,2,1.508,0.0,--| Possible [OII] in skyline regions|--,3,none
90,35185772164286546,GALAXY,1.508,TWL,2.0,--,--,--,Possible [OII] in skyline regions,1.508,GALAXY,2.666667,2,1.508,0.0,--| Possible [OII] in skyline regions|--,3,none
140,35185772164286546,GALAXY,1.508,ACE,4.0,--,--,--,--,1.508,GALAXY,2.666667,2,1.508,0.0,--| Possible [OII] in skyline regions|--,3,none


In [51]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. z not secure, set to 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
40,35185772164286546,-1,0.33.0.dev3890,0,GALAXY,1.508,HSH,2.0,--,--,...,105.410089,1.508,GALAXY,2.666667,2,0.0,1.508,--| Possible [OII] in skyline regions|--,3,"flag disagreement. z not secure, set to 2."
90,35185772164286546,-1,0.33.0.dev3890,0,GALAXY,1.508,TWL,2.0,--,--,...,105.410089,1.508,GALAXY,2.666667,2,0.0,1.508,--| Possible [OII] in skyline regions|--,3,"flag disagreement. z not secure, set to 2."
140,35185772164286546,-1,0.33.0.dev3890,0,GALAXY,1.508,ACE,2.0,--,--,...,105.410089,1.508,GALAXY,2.666667,2,0.0,1.508,--| Possible [OII] in skyline regions|--,3,"flag disagreement. z not secure, set to 2."


conflict ID =  15


In [52]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
49,35185772164286597,GALAXY,0.7307,HSH,1.0,R,--,--,--,0.7307,GALAXY,1.333333,3,0.7307,0.7307,--| Low S/N|--,3,none
99,35185772164286597,GALAXY,0.7307,TWL,0.0,--,--,--,Low S/N,0.7307,GALAXY,1.333333,3,0.7307,0.7307,--| Low S/N|--,3,none
149,35185772164286597,GALAXY,0.7307,ACE,3.0,--,0.0000,STAR,--,0.0000,STAR,1.333333,3,0.7307,0.7307,--| Low S/N|--,3,none


In [53]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 0.7307 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag and spectype disagreement. too low SNR. take rr answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
49,35185772164286597,-1,0.33.0.dev3890,0,GALAXY,0.7307,HSH,0.7307,R,--,...,10.792331,1.0,GALAXY,1.333333,3,0.7307,0.7307,--| Low S/N|--,3,flag and spectype disagreement. too low SNR. t...
99,35185772164286597,-1,0.33.0.dev3890,0,GALAXY,0.7307,TWL,0.7307,--,--,...,10.792331,1.0,GALAXY,1.333333,3,0.7307,0.7307,--| Low S/N|--,3,flag and spectype disagreement. too low SNR. t...
149,35185772164286597,-1,0.33.0.dev3890,0,GALAXY,0.7307,ACE,0.7307,--,0.0000,...,10.792331,1.0,GALAXY,1.333333,3,0.7307,0.7307,--| Low S/N|--,3,flag and spectype disagreement. too low SNR. t...


conflict ID =  16


In [54]:
#copy this text to a new cell to display the conflict
conflict_id=17
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
43,35185772164288083,GALAXY,1.5409,HSH,0.0,R,--,--,--,1.5409,GALAXY,1.666667,3,1.5409,0.0,--| Possible [OII] emission in skyline regions...,3,none
93,35185772164288083,GALAXY,1.5409,TWL,2.0,--,--,--,Possible [OII] emission in skyline regions,1.5409,GALAXY,1.666667,3,1.5409,0.0,--| Possible [OII] emission in skyline regions...,3,none
143,35185772164288083,GALAXY,1.5409,ACE,3.0,--,--,--,OII and CIV,1.5409,GALAXY,1.666667,3,1.5409,0.0,--| Possible [OII] emission in skyline regions...,3,none


In [55]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
43,35185772164288083,-1,0.33.0.dev3890,0,GALAXY,1.5409,HSH,1.666667,R,--,...,92.73525,1.5409,GALAXY,1.666667,3,0.0,1.5409,--| Possible [OII] emission in skyline regions...,3,"flag disagreement, take mean."
93,35185772164288083,-1,0.33.0.dev3890,0,GALAXY,1.5409,TWL,1.666667,--,--,...,92.73525,1.5409,GALAXY,1.666667,3,0.0,1.5409,--| Possible [OII] emission in skyline regions...,3,"flag disagreement, take mean."
143,35185772164288083,-1,0.33.0.dev3890,0,GALAXY,1.5409,ACE,1.666667,--,--,...,92.73525,1.5409,GALAXY,1.666667,3,0.0,1.5409,--| Possible [OII] emission in skyline regions...,3,"flag disagreement, take mean."


conflict ID =  17


In [56]:
#copy this text to a new cell to display the conflict
conflict_id=18
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
44,35185772164288175,GALAXY,1.2869,HSH,2.0,--,--,--,--,1.2869,GALAXY,3.333333,2,1.2869,0.0,--|--| CIV in absorption,3,none
94,35185772164288175,GALAXY,1.2869,TWL,4.0,--,--,--,--,1.2869,GALAXY,3.333333,2,1.2869,0.0,--|--| CIV in absorption,3,none
144,35185772164288175,GALAXY,1.2869,ACE,4.0,--,--,--,CIV in absorption,1.2869,GALAXY,3.333333,2,1.2869,0.0,--|--| CIV in absorption,3,none


In [57]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean, z secure. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
44,35185772164288175,-1,0.33.0.dev3890,0,GALAXY,1.2869,HSH,3.333333,--,--,...,17.749891,1.2869,GALAXY,3.333333,2,0.0,1.2869,--|--| CIV in absorption,3,"flag disagreement, take mean, z secure. poor s..."
94,35185772164288175,-1,0.33.0.dev3890,0,GALAXY,1.2869,TWL,3.333333,--,--,...,17.749891,1.2869,GALAXY,3.333333,2,0.0,1.2869,--|--| CIV in absorption,3,"flag disagreement, take mean, z secure. poor s..."
144,35185772164288175,-1,0.33.0.dev3890,0,GALAXY,1.2869,ACE,3.333333,--,--,...,17.749891,1.2869,GALAXY,3.333333,2,0.0,1.2869,--|--| CIV in absorption,3,"flag disagreement, take mean, z secure. poor s..."


conflict ID =  18


In [58]:
#copy this text to a new cell to display the conflict
conflict_id=19
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
151,35185772164289259,GALAXY,0.9955,TWL,2.0,--,--,--,Possible [OII] emission next to sky lines wit...,0.9955,GALAXY,3.0,2,0.995633,0.0002,Possible [OII] emission next to sky lines wit...,3,none
201,35185772164289259,GALAXY,0.9955,ACE,4.0,--,0.9959,GALAXY,MgII and CIII] in emission,0.9959,GALAXY,3.0,2,0.995633,0.0002,Possible [OII] emission next to sky lines wit...,3,none
251,35185772164289259,GALAXY,0.9955,BJW,3.0,--,--,--,--,0.9955,GALAXY,3.0,2,0.995633,0.0002,Possible [OII] emission next to sky lines wit...,3,none


In [59]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. unresolved OII and weak balmer lines.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
151,35185772164289259,-1,0.33.0.dev3890,0,GALAXY,0.9955,TWL,3.0,--,--,...,17.937318,0.9955,GALAXY,3.0,2,0.0002,0.995633,Possible [OII] emission next to sky lines wit...,3,"flag disagreement, take mean. unresolved OII a..."
201,35185772164289259,-1,0.33.0.dev3890,0,GALAXY,0.9955,ACE,3.0,--,0.9959,...,17.937318,0.9959,GALAXY,3.0,2,0.0002,0.995633,Possible [OII] emission next to sky lines wit...,3,"flag disagreement, take mean. unresolved OII a..."
251,35185772164289259,-1,0.33.0.dev3890,0,GALAXY,0.9955,BJW,3.0,--,--,...,17.937318,0.9955,GALAXY,3.0,2,0.0002,0.995633,Possible [OII] emission next to sky lines wit...,3,"flag disagreement, take mean. unresolved OII a..."


conflict ID =  19


In [60]:
#copy this text to a new cell to display the conflict
conflict_id=20
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
171,35185772168481893,GALAXY,1.4806,TWL,1.0,RC,--,STAR,Likely a white dwarf; DR8 indicates it is a p...,1.4806,STAR,1.333333,3,1.4806,1.484326,Likely a white dwarf; DR8 indicates it is a p...,3,none
221,35185772168481893,GALAXY,1.4806,ACE,3.0,RC,-0.0015,STAR,White dwarf.,-0.0015,STAR,1.333333,3,1.4806,1.484326,Likely a white dwarf; DR8 indicates it is a p...,3,none
271,35185772168481893,GALAXY,1.4806,BJW,0.0,R,--,--,--,1.4806,GALAXY,1.333333,3,1.4806,1.484326,Likely a white dwarf; DR8 indicates it is a p...,3,none


In [61]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'STAR'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and spectype. 2/3 inspectors suggest WD. take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
171,35185772168481893,-1,0.33.0.dev3890,0,GALAXY,1.4806,TWL,2.0,RC,--,...,14.330362,0.0,STAR,1.333333,3,1.484326,1.4806,Likely a white dwarf; DR8 indicates it is a p...,3,disagreement in flag and spectype. 2/3 inspect...
221,35185772168481893,-1,0.33.0.dev3890,0,GALAXY,1.4806,ACE,2.0,RC,-0.0015,...,14.330362,0.0,STAR,1.333333,3,1.484326,1.4806,Likely a white dwarf; DR8 indicates it is a p...,3,disagreement in flag and spectype. 2/3 inspect...
271,35185772168481893,-1,0.33.0.dev3890,0,GALAXY,1.4806,BJW,2.0,R,--,...,14.330362,0.0,STAR,1.333333,3,1.484326,1.4806,Likely a white dwarf; DR8 indicates it is a p...,3,disagreement in flag and spectype. 2/3 inspect...


conflict ID =  20


In [62]:
#copy this text to a new cell to display the conflict
conflict_id=21
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
173,35185772168481920,GALAXY,1.3839,TWL,2.0,--,--,--,Possible [OII] next to sky lines; being conse...,1.3839,GALAXY,3.0,2,1.3839,0.0,Possible [OII] next to sky lines; being conse...,3,none
223,35185772168481920,GALAXY,1.3839,ACE,4.0,--,--,--,--,1.3839,GALAXY,3.0,2,1.3839,0.0,Possible [OII] next to sky lines; being conse...,3,none
273,35185772168481920,GALAXY,1.3839,BJW,3.0,--,--,--,--,1.3839,GALAXY,3.0,2,1.3839,0.0,Possible [OII] next to sky lines; being conse...,3,none


In [63]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. doublet not fully resolved due to skyline. slight change in continuum consistent with OII'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
173,35185772168481920,-1,0.33.0.dev3890,0,GALAXY,1.3839,TWL,3.0,--,--,...,179.825273,1.3839,GALAXY,3.0,2,0.0,1.3839,Possible [OII] next to sky lines; being conse...,3,"flag disagreement, take mean. doublet not full..."
223,35185772168481920,-1,0.33.0.dev3890,0,GALAXY,1.3839,ACE,3.0,--,--,...,179.825273,1.3839,GALAXY,3.0,2,0.0,1.3839,Possible [OII] next to sky lines; being conse...,3,"flag disagreement, take mean. doublet not full..."
273,35185772168481920,-1,0.33.0.dev3890,0,GALAXY,1.3839,BJW,3.0,--,--,...,179.825273,1.3839,GALAXY,3.0,2,0.0,1.3839,Possible [OII] next to sky lines; being conse...,3,"flag disagreement, take mean. doublet not full..."


conflict ID =  21


In [64]:
#copy this text to a new cell to display the conflict
conflict_id=22
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
176,35185772172673676,GALAXY,1.039,TWL,2.0,--,--,--,[OII] emission on sky lines; being conservative,1.039,GALAXY,3.0,2,1.039,0.0,[OII] emission on sky lines; being conservati...,3,none
226,35185772172673676,GALAXY,1.039,ACE,4.0,--,--,--,--,1.039,GALAXY,3.0,2,1.039,0.0,[OII] emission on sky lines; being conservati...,3,none
276,35185772172673676,GALAXY,1.039,BJW,3.0,--,--,--,--,1.039,GALAXY,3.0,2,1.039,0.0,[OII] emission on sky lines; being conservati...,3,none


In [65]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. doublet entirely on skyline, not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
176,35185772172673676,-1,0.33.0.dev3890,0,GALAXY,1.039,TWL,2.0,--,--,...,60.675695,1.039,GALAXY,3.0,2,0.0,1.039,[OII] emission on sky lines; being conservati...,3,disagreement in flag. doublet entirely on skyl...
226,35185772172673676,-1,0.33.0.dev3890,0,GALAXY,1.039,ACE,2.0,--,--,...,60.675695,1.039,GALAXY,3.0,2,0.0,1.039,[OII] emission on sky lines; being conservati...,3,disagreement in flag. doublet entirely on skyl...
276,35185772172673676,-1,0.33.0.dev3890,0,GALAXY,1.039,BJW,2.0,--,--,...,60.675695,1.039,GALAXY,3.0,2,0.0,1.039,[OII] emission on sky lines; being conservati...,3,disagreement in flag. doublet entirely on skyl...


conflict ID =  22


In [66]:
#copy this text to a new cell to display the conflict
conflict_id=23
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
178,35185772172673848,GALAXY,0.8102,TWL,2.0,--,1.4316,--,Possible [OII] at 9060,1.4316,GALAXY,1.666667,3,0.8102,0.343277,Possible [OII] at 9060| Redrock confuses OII ...,3,none
228,35185772172673848,GALAXY,0.8102,ACE,3.0,R,1.4316,GALAXY,Redrock confuses OII for OIII,1.4316,GALAXY,1.666667,3,0.8102,0.343277,Possible [OII] at 9060| Redrock confuses OII ...,3,none
278,35185772172673848,GALAXY,0.8102,BJW,0.0,R,--,--,--,0.8102,GALAXY,1.666667,3,0.8102,0.343277,Possible [OII] at 9060| Redrock confuses OII ...,3,none


In [67]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4316
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/3 inspectors give same VI z based on doublet missed by RR. take with their mean flag of 2.5.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
178,35185772172673848,-1,0.33.0.dev3890,0,GALAXY,0.8102,TWL,2.5,--,1.4316,...,6.7314,1.4316,GALAXY,1.666667,3,0.343277,0.8102,Possible [OII] at 9060| Redrock confuses OII ...,3,disagreement in flag and z. 2/3 inspectors giv...
228,35185772172673848,-1,0.33.0.dev3890,0,GALAXY,0.8102,ACE,2.5,R,1.4316,...,6.7314,1.4316,GALAXY,1.666667,3,0.343277,0.8102,Possible [OII] at 9060| Redrock confuses OII ...,3,disagreement in flag and z. 2/3 inspectors giv...
278,35185772172673848,-1,0.33.0.dev3890,0,GALAXY,0.8102,BJW,2.5,R,--,...,6.7314,1.4316,GALAXY,1.666667,3,0.343277,0.8102,Possible [OII] at 9060| Redrock confuses OII ...,3,disagreement in flag and z. 2/3 inspectors giv...


conflict ID =  23


In [68]:
#copy this text to a new cell to display the conflict
conflict_id=24
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
181,35185772172674321,GALAXY,1.0746,TWL,3.0,--,--,--,Some absorption features,1.0746,GALAXY,1.666667,3,1.0746,1.0746,Some absorption features|Possible star| Redsh...,3,none
231,35185772172674321,GALAXY,1.0746,ACE,2.0,RC,0,STAR,Possible star,0.0000,STAR,1.666667,3,1.0746,1.0746,Some absorption features|Possible star| Redsh...,3,none
281,35185772172674321,GALAXY,1.0746,BJW,0.0,R,--,--,Redshift fit not real. Low S/N for the magnit...,1.0746,GALAXY,1.666667,3,1.0746,1.0746,Some absorption features|Possible star| Redsh...,3,none


In [69]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'STAR'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'no strong features. 1/3 suggests star. take with their flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
181,35185772172674321,-1,0.33.0.dev3890,0,GALAXY,1.0746,TWL,2.0,--,--,...,80.560709,0.0,STAR,1.666667,3,1.0746,1.0746,Some absorption features|Possible star| Redsh...,3,no strong features. 1/3 suggests star. take wi...
231,35185772172674321,-1,0.33.0.dev3890,0,GALAXY,1.0746,ACE,2.0,RC,0,...,80.560709,0.0,STAR,1.666667,3,1.0746,1.0746,Some absorption features|Possible star| Redsh...,3,no strong features. 1/3 suggests star. take wi...
281,35185772172674321,-1,0.33.0.dev3890,0,GALAXY,1.0746,BJW,2.0,R,--,...,80.560709,0.0,STAR,1.666667,3,1.0746,1.0746,Some absorption features|Possible star| Redsh...,3,no strong features. 1/3 suggests star. take wi...


conflict ID =  24


In [70]:
#copy this text to a new cell to display the conflict
conflict_id=25
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
199,35185772172676812,GALAXY,1.0468,TWL,2.0,--,--,--,Possible [OII] emission in skyline region,1.0468,GALAXY,2.0,2,1.0468,0.0,Possible [OII] emission in skyline region|No ...,3,none
249,35185772172676812,GALAXY,1.0468,ACE,1.0,--,--,--,No clear match to features.,1.0468,GALAXY,2.0,2,1.0468,0.0,Possible [OII] emission in skyline region|No ...,3,none
299,35185772172676812,GALAXY,1.0468,BJW,3.0,--,--,--,--,1.0468,GALAXY,2.0,2,1.0468,0.0,Possible [OII] emission in skyline region|No ...,3,none


In [71]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
199,35185772172676812,-1,0.33.0.dev3890,0,GALAXY,1.0468,TWL,2.0,--,--,...,20.914977,1.0468,GALAXY,2.0,2,0.0,1.0468,Possible [OII] emission in skyline region|No ...,3,"flag disagreement, take mean."
249,35185772172676812,-1,0.33.0.dev3890,0,GALAXY,1.0468,ACE,2.0,--,--,...,20.914977,1.0468,GALAXY,2.0,2,0.0,1.0468,Possible [OII] emission in skyline region|No ...,3,"flag disagreement, take mean."
299,35185772172676812,-1,0.33.0.dev3890,0,GALAXY,1.0468,BJW,2.0,--,--,...,20.914977,1.0468,GALAXY,2.0,2,0.0,1.0468,Possible [OII] emission in skyline region|No ...,3,"flag disagreement, take mean."


conflict ID =  25


In [72]:
#copy this text to a new cell to display the conflict
conflict_id=26
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
300,35185772172677547,GALAXY,1.4112,ACE,4.0,--,--,--,MgII in absorption,1.4112,GALAXY,2.666667,2,1.4112,0.0,MgII in absorption|Possible [OII] emission nex...,3,none
350,35185772172677547,GALAXY,1.4112,TWL,2.0,--,--,--,Possible [OII] emission next to sky lines; bei...,1.4112,GALAXY,2.666667,2,1.4112,0.0,MgII in absorption|Possible [OII] emission nex...,3,none
400,35185772172677547,GALAXY,1.4112,BJW,2.0,--,--,--,[O II] on a skyline,1.4112,GALAXY,2.666667,2,1.4112,0.0,MgII in absorption|Possible [OII] emission nex...,3,none


In [73]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag,z not secure. bad sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
300,35185772172677547,-1,0.33.0.dev3890,0,GALAXY,1.4112,ACE,2.0,--,--,...,26.208257,1.4112,GALAXY,2.666667,2,0.0,1.4112,MgII in absorption|Possible [OII] emission nex...,3,"disagreement in flag,z not secure. bad sky sub..."
350,35185772172677547,-1,0.33.0.dev3890,0,GALAXY,1.4112,TWL,2.0,--,--,...,26.208257,1.4112,GALAXY,2.666667,2,0.0,1.4112,MgII in absorption|Possible [OII] emission nex...,3,"disagreement in flag,z not secure. bad sky sub..."
400,35185772172677547,-1,0.33.0.dev3890,0,GALAXY,1.4112,BJW,2.0,--,--,...,26.208257,1.4112,GALAXY,2.666667,2,0.0,1.4112,MgII in absorption|Possible [OII] emission nex...,3,"disagreement in flag,z not secure. bad sky sub..."


conflict ID =  26


In [74]:
#copy this text to a new cell to display the conflict
conflict_id=27
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
318,35185772176869420,GALAXY,0.5301,ACE,3.0,R,0.4345,GALAXY,Based on OII and NeIII,0.4345,GALAXY,1.0,3,0.5301,0.066643,Based on OII and NeIII| Low S/N|--,3,none
368,35185772176869420,GALAXY,0.5301,TWL,0.0,--,--,--,Low S/N,0.5301,GALAXY,1.0,3,0.5301,0.066643,Based on OII and NeIII| Low S/N|--,3,none
418,35185772176869420,GALAXY,0.5301,BJW,0.0,R,--,--,--,0.5301,GALAXY,1.0,3,0.5301,0.066643,Based on OII and NeIII| Low S/N|--,3,none


In [75]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5301
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. low SNR. take rr answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
318,35185772176869420,-1,0.33.0.dev3890,0,GALAXY,0.5301,ACE,1.0,R,0.4345,...,1.174251,0.5301,GALAXY,1.0,3,0.066643,0.5301,Based on OII and NeIII| Low S/N|--,3,disagreement in flag and z. low SNR. take rr a...
368,35185772176869420,-1,0.33.0.dev3890,0,GALAXY,0.5301,TWL,1.0,--,--,...,1.174251,0.5301,GALAXY,1.0,3,0.066643,0.5301,Based on OII and NeIII| Low S/N|--,3,disagreement in flag and z. low SNR. take rr a...
418,35185772176869420,-1,0.33.0.dev3890,0,GALAXY,0.5301,BJW,1.0,R,--,...,1.174251,0.5301,GALAXY,1.0,3,0.066643,0.5301,Based on OII and NeIII| Low S/N|--,3,disagreement in flag and z. low SNR. take rr a...


conflict ID =  27


In [76]:
#copy this text to a new cell to display the conflict
conflict_id=28
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
304,35185772176871722,QSO,4.8054,ACE,4.0,RC,1.2180,GALAXY,"Continuum """"dip"""" 6800-7300A.",1.2180,GALAXY,3.0,2,1.217933,0.00009,"Continuum """"dip"""" 6800-7300A.| Calibration f...",3,none
354,35185772176871722,QSO,4.8054,TWL,3.0,RCS,1.2178,GALAXY,Calibration features around 7000 A; possible ...,1.2178,GALAXY,3.0,2,1.217933,0.00009,"Continuum """"dip"""" 6800-7300A.| Calibration f...",3,none
404,35185772176871722,QSO,4.8054,BJW,2.0,RCS,1.2180,GALAXY,Bad continuum dip fit by inverted QSO. Possibl...,1.2180,GALAXY,3.0,2,1.217933,0.00009,"Continuum """"dip"""" 6800-7300A.| Calibration f...",3,none


In [77]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2180
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but all 3/3 inspectors agree on VI z. take with mean flag 3. not a QSO. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
304,35185772176871722,-1,0.33.0.dev3890,0,QSO,4.8054,ACE,2.0,RC,1.2180,...,63.332511,1.218,GALAXY,3.0,2,0.00009,1.217933,"Continuum """"dip"""" 6800-7300A.| Calibration f...",3,"disagreement in flag, but all 3/3 inspectors a..."
354,35185772176871722,-1,0.33.0.dev3890,0,QSO,4.8054,TWL,2.0,RCS,1.2178,...,63.332511,1.218,GALAXY,3.0,2,0.00009,1.217933,"Continuum """"dip"""" 6800-7300A.| Calibration f...",3,"disagreement in flag, but all 3/3 inspectors a..."
404,35185772176871722,-1,0.33.0.dev3890,0,QSO,4.8054,BJW,2.0,RCS,1.2180,...,63.332511,1.218,GALAXY,3.0,2,0.00009,1.217933,"Continuum """"dip"""" 6800-7300A.| Calibration f...",3,"disagreement in flag, but all 3/3 inspectors a..."


conflict ID =  28


In [78]:
#copy this text to a new cell to display the conflict
conflict_id=29
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
335,35185772181062339,GALAXY,0.7814,ACE,3.0,R,1.5755,GALAXY,Clear OII in sky lines.,1.5755,GALAXY,1.333333,3,0.7814,0.445773,Clear OII in sky lines.| Model fits sky resid...,3,none
385,35185772181062339,GALAXY,0.7814,TWL,1.0,--,--,--,Model fits sky residuals,0.7814,GALAXY,1.333333,3,0.7814,0.445773,Clear OII in sky lines.| Model fits sky resid...,3,none
435,35185772181062339,GALAXY,0.7814,BJW,0.0,--,--,--,--,0.7814,GALAXY,1.333333,3,0.7814,0.445773,Clear OII in sky lines.| Model fits sky resid...,3,none


In [79]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5757
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 1/3 inspector gives VI z based on OII doublet missed by RR. secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
335,35185772181062339,-1,0.33.0.dev3890,0,GALAXY,0.7814,ACE,3.0,R,1.5755,...,2.509822,1.5757,GALAXY,1.333333,3,0.445773,0.7814,Clear OII in sky lines.| Model fits sky resid...,3,disagreement in flag and z. 1/3 inspector give...
385,35185772181062339,-1,0.33.0.dev3890,0,GALAXY,0.7814,TWL,3.0,--,--,...,2.509822,1.5757,GALAXY,1.333333,3,0.445773,0.7814,Clear OII in sky lines.| Model fits sky resid...,3,disagreement in flag and z. 1/3 inspector give...
435,35185772181062339,-1,0.33.0.dev3890,0,GALAXY,0.7814,BJW,3.0,--,--,...,2.509822,1.5757,GALAXY,1.333333,3,0.445773,0.7814,Clear OII in sky lines.| Model fits sky resid...,3,disagreement in flag and z. 1/3 inspector give...


conflict ID =  29


In [82]:
#copy this text to a new cell to display the conflict
conflict_id=30
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
343,35185772181063930,GALAXY,0.0008,ACE,-1.0,C,0.0008,STAR,White dwarf.,0.0008,STAR,2.333333,5,0.0008,0.0,White dwarf.| A star; DR8 indicates it as a p...,3,none
393,35185772181063930,GALAXY,0.0008,TWL,4.0,RC,--,STAR,A star; DR8 indicates it as a point source,0.0008,STAR,2.333333,5,0.0008,0.0,White dwarf.| A star; DR8 indicates it as a p...,3,none
443,35185772181063930,GALAXY,0.0008,BJW,4.0,C,--,STAR,fit emission line galaxy to hot star / A star,0.0008,STAR,2.333333,5,0.0008,0.0,White dwarf.| A star; DR8 indicates it as a p...,3,none


In [83]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'STAR'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'WD STAR classified as a GALAXY. One inspector forgot to set a flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
343,35185772181063930,-1,0.33.0.dev3890,0,GALAXY,0.0008,ACE,4.0,C,0.0008,...,124.436233,0.0008,STAR,2.333333,5,0.0,0.0008,White dwarf.| A star; DR8 indicates it as a p...,3,WD STAR classified as a GALAXY. One inspector ...
393,35185772181063930,-1,0.33.0.dev3890,0,GALAXY,0.0008,TWL,4.0,RC,--,...,124.436233,0.0008,STAR,2.333333,5,0.0,0.0008,White dwarf.| A star; DR8 indicates it as a p...,3,WD STAR classified as a GALAXY. One inspector ...
443,35185772181063930,-1,0.33.0.dev3890,0,GALAXY,0.0008,BJW,4.0,C,--,...,124.436233,0.0008,STAR,2.333333,5,0.0,0.0008,White dwarf.| A star; DR8 indicates it as a p...,3,WD STAR classified as a GALAXY. One inspector ...


conflict ID =  30


In [86]:
#copy this text to a new cell to display the conflict
conflict_id=31
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
464,35185772181065428,GALAXY,1.0723,ACE,4.0,--,--,--,--,1.0723,GALAXY,3.0,2,1.0723,0.0,--|--| Broad [OII] emission; Calibration featu...,3,none
513,35185772181065428,GALAXY,1.0723,DRP,2.0,--,--,--,--,1.0723,GALAXY,3.0,2,1.0723,0.0,--|--| Broad [OII] emission; Calibration featu...,3,none
563,35185772181065428,GALAXY,1.0723,TWL,3.0,--,--,--,Broad [OII] emission; Calibration features ar...,1.0723,GALAXY,3.0,2,1.0723,0.0,--|--| Broad [OII] emission; Calibration featu...,3,none


In [87]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. broadened but unresolved line. change in continuum consistent with OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
464,35185772181065428,-1,0.33.0.dev3890,0,GALAXY,1.0723,ACE,3.0,--,--,...,108.810026,1.0723,GALAXY,3.0,2,0.0,1.0723,--|--| Broad [OII] emission; Calibration featu...,3,"flag disagreement, take mean. broadened but un..."
513,35185772181065428,-1,0.33.0.dev3890,0,GALAXY,1.0723,DRP,3.0,--,--,...,108.810026,1.0723,GALAXY,3.0,2,0.0,1.0723,--|--| Broad [OII] emission; Calibration featu...,3,"flag disagreement, take mean. broadened but un..."
563,35185772181065428,-1,0.33.0.dev3890,0,GALAXY,1.0723,TWL,3.0,--,--,...,108.810026,1.0723,GALAXY,3.0,2,0.0,1.0723,--|--| Broad [OII] emission; Calibration featu...,3,"flag disagreement, take mean. broadened but un..."


conflict ID =  31


In [90]:
#copy this text to a new cell to display the conflict
conflict_id=32
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
465,35185772181065499,GALAXY,1.1837,ACE,4.0,--,--,--,MgII in emission,1.1837,GALAXY,3.333333,2,1.1837,0.0,MgII in emission|--|--,3,none
514,35185772181065499,GALAXY,1.1837,DRP,2.0,--,--,--,--,1.1837,GALAXY,3.333333,2,1.1837,0.0,MgII in emission|--|--,3,none
564,35185772181065499,GALAXY,1.1837,TWL,4.0,--,--,--,--,1.1837,GALAXY,3.333333,2,1.1837,0.0,MgII in emission|--|--,3,none


In [91]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line. no continuum. no other features. z not secure. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
465,35185772181065499,-1,0.33.0.dev3890,0,GALAXY,1.1837,ACE,2.0,--,--,...,54.002232,1.1837,GALAXY,3.333333,2,0.0,1.1837,MgII in emission|--|--,3,disagreement in flag. broadened but unresolved...
514,35185772181065499,-1,0.33.0.dev3890,0,GALAXY,1.1837,DRP,2.0,--,--,...,54.002232,1.1837,GALAXY,3.333333,2,0.0,1.1837,MgII in emission|--|--,3,disagreement in flag. broadened but unresolved...
564,35185772181065499,-1,0.33.0.dev3890,0,GALAXY,1.1837,TWL,2.0,--,--,...,54.002232,1.1837,GALAXY,3.333333,2,0.0,1.1837,MgII in emission|--|--,3,disagreement in flag. broadened but unresolved...


conflict ID =  32


In [92]:
#copy this text to a new cell to display the conflict
conflict_id=33
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
468,35185772185256183,GALAXY,0.8237,ACE,3.0,--,2.5230,QSO,Strong narrow Lyman alpha,2.5230,QSO,1.666667,2,0.8237,0.931787,Strong narrow Lyman alpha|--|Low S/N,3,none
517,35185772185256183,GALAXY,0.8237,DRP,1.0,--,--,--,--,0.8237,GALAXY,1.666667,2,0.8237,0.931787,Strong narrow Lyman alpha|--|Low S/N,3,none
567,35185772185256183,GALAXY,0.8237,TWL,1.0,--,--,--,Low S/N,0.8237,GALAXY,1.666667,2,0.8237,0.931787,Strong narrow Lyman alpha|--|Low S/N,3,none


In [93]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8237
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 1/3 inspectors suggests QSO based on narrow LyA. take RR answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
468,35185772185256183,-1,0.33.0.dev3890,0,GALAXY,0.8237,ACE,1.0,--,2.5230,...,9.868241,0.8237,GALAXY,1.666667,2,0.931787,0.8237,Strong narrow Lyman alpha|--|Low S/N,3,disagreement in flag and z. 1/3 inspectors sug...
517,35185772185256183,-1,0.33.0.dev3890,0,GALAXY,0.8237,DRP,1.0,--,--,...,9.868241,0.8237,GALAXY,1.666667,2,0.931787,0.8237,Strong narrow Lyman alpha|--|Low S/N,3,disagreement in flag and z. 1/3 inspectors sug...
567,35185772185256183,-1,0.33.0.dev3890,0,GALAXY,0.8237,TWL,1.0,--,--,...,9.868241,0.8237,GALAXY,1.666667,2,0.931787,0.8237,Strong narrow Lyman alpha|--|Low S/N,3,disagreement in flag and z. 1/3 inspectors sug...


conflict ID =  33


In [96]:
#copy this text to a new cell to display the conflict
conflict_id=34
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
469,35185772185256282,GALAXY,1.2043,ACE,4.0,--,--,--,--,1.2043,GALAXY,3.333333,2,1.2043,0.0,--|--|Calibration features around 5600 A,3,none
518,35185772185256282,GALAXY,1.2043,DRP,2.0,--,--,--,--,1.2043,GALAXY,3.333333,2,1.2043,0.0,--|--|Calibration features around 5600 A,3,none
568,35185772185256282,GALAXY,1.2043,TWL,4.0,S,--,--,Calibration features around 5600 A,1.2043,GALAXY,3.333333,2,1.2043,0.0,--|--|Calibration features around 5600 A,3,none


In [98]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. doublet secures z'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
469,35185772185256282,-1,0.33.0.dev3890,0,GALAXY,1.2043,ACE,3.333333,--,--,...,56.540733,1.2043,GALAXY,3.333333,2,0.0,1.2043,--|--|Calibration features around 5600 A,3,"flag disagreement, take mean. doublet secures z"
518,35185772185256282,-1,0.33.0.dev3890,0,GALAXY,1.2043,DRP,3.333333,--,--,...,56.540733,1.2043,GALAXY,3.333333,2,0.0,1.2043,--|--|Calibration features around 5600 A,3,"flag disagreement, take mean. doublet secures z"
568,35185772185256282,-1,0.33.0.dev3890,0,GALAXY,1.2043,TWL,3.333333,S,--,...,56.540733,1.2043,GALAXY,3.333333,2,0.0,1.2043,--|--|Calibration features around 5600 A,3,"flag disagreement, take mean. doublet secures z"


conflict ID =  34


In [101]:
#copy this text to a new cell to display the conflict
conflict_id=35
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
470,35185772185256510,GALAXY,1.5812,ACE,4.0,--,--,--,--,1.5812,GALAXY,3.0,2,1.5812,0.0,--|--| Possible broad [OII] on top of sky lines,3,none
519,35185772185256510,GALAXY,1.5812,DRP,2.0,--,--,--,--,1.5812,GALAXY,3.0,2,1.5812,0.0,--|--| Possible broad [OII] on top of sky lines,3,none
569,35185772185256510,GALAXY,1.5812,TWL,3.0,--,--,--,Possible broad [OII] on top of sky lines,1.5812,GALAXY,3.0,2,1.5812,0.0,--|--| Possible broad [OII] on top of sky lines,3,none


In [102]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line on skyline region. no continuum or other features. not secure. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
470,35185772185256510,-1,0.33.0.dev3890,0,GALAXY,1.5812,ACE,2.0,--,--,...,54.721394,1.5812,GALAXY,3.0,2,0.0,1.5812,--|--| Possible broad [OII] on top of sky lines,3,disagreement in flag. broadened but unresolved...
519,35185772185256510,-1,0.33.0.dev3890,0,GALAXY,1.5812,DRP,2.0,--,--,...,54.721394,1.5812,GALAXY,3.0,2,0.0,1.5812,--|--| Possible broad [OII] on top of sky lines,3,disagreement in flag. broadened but unresolved...
569,35185772185256510,-1,0.33.0.dev3890,0,GALAXY,1.5812,TWL,2.0,--,--,...,54.721394,1.5812,GALAXY,3.0,2,0.0,1.5812,--|--| Possible broad [OII] on top of sky lines,3,disagreement in flag. broadened but unresolved...


conflict ID =  35


In [105]:
#copy this text to a new cell to display the conflict
conflict_id=36
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
526,35185772185257394,GALAXY,1.1458,DRP,2.0,--,--,--,--,1.1458,GALAXY,3.0,2,1.1458,0.0,--|--,2,none
576,35185772185257394,GALAXY,1.1458,TWL,4.0,--,--,--,--,1.1458,GALAXY,3.0,2,1.1458,0.0,--|--,2,none


In [106]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
526,35185772185257394,-1,0.33.0.dev3890,0,GALAXY,1.1458,DRP,3.0,--,--,...,76.583839,1.1458,GALAXY,3.0,2,0.0,1.1458,--|--,2,"flag disagreement, take mean. doublet secures z."
576,35185772185257394,-1,0.33.0.dev3890,0,GALAXY,1.1458,TWL,3.0,--,--,...,76.583839,1.1458,GALAXY,3.0,2,0.0,1.1458,--|--,2,"flag disagreement, take mean. doublet secures z."


conflict ID =  36


In [110]:
#copy this text to a new cell to display the conflict
conflict_id=37
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
477,35185772185257398,GALAXY,1.3997,ACE,3.0,--,--,--,--,1.3997,GALAXY,2.333333,2,1.3997,0.0,--|--| Possible [OII] emission on top of sky l...,3,none
527,35185772185257398,GALAXY,1.3997,DRP,1.0,--,--,--,--,1.3997,GALAXY,2.333333,2,1.3997,0.0,--|--| Possible [OII] emission on top of sky l...,3,none
577,35185772185257398,GALAXY,1.3997,TWL,3.0,--,--,--,Possible [OII] emission on top of sky lines,1.3997,GALAXY,2.333333,2,1.3997,0.0,--|--| Possible [OII] emission on top of sky l...,3,none


In [111]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. line in skylines, not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
477,35185772185257398,-1,0.33.0.dev3890,0,GALAXY,1.3997,ACE,2.333333,--,--,...,36.195199,1.3997,GALAXY,2.333333,2,0.0,1.3997,--|--| Possible [OII] emission on top of sky l...,3,"flag disagreement, take mean. line in skylines..."
527,35185772185257398,-1,0.33.0.dev3890,0,GALAXY,1.3997,DRP,2.333333,--,--,...,36.195199,1.3997,GALAXY,2.333333,2,0.0,1.3997,--|--| Possible [OII] emission on top of sky l...,3,"flag disagreement, take mean. line in skylines..."
577,35185772185257398,-1,0.33.0.dev3890,0,GALAXY,1.3997,TWL,2.333333,--,--,...,36.195199,1.3997,GALAXY,2.333333,2,0.0,1.3997,--|--| Possible [OII] emission on top of sky l...,3,"flag disagreement, take mean. line in skylines..."


conflict ID =  37


In [114]:
#copy this text to a new cell to display the conflict
conflict_id=38
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
479,35185772185257520,GALAXY,0.9893,ACE,4.0,--,--,--,--,0.9893,GALAXY,3.333333,2,0.9893,0.0,--|--|--,3,none
529,35185772185257520,GALAXY,0.9893,DRP,2.0,--,--,--,--,0.9893,GALAXY,3.333333,2,0.9893,0.0,--|--|--,3,none
579,35185772185257520,GALAXY,0.9893,TWL,4.0,--,--,--,--,0.9893,GALAXY,3.333333,2,0.9893,0.0,--|--|--,3,none


In [115]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
479,35185772185257520,-1,0.33.0.dev3890,0,GALAXY,0.9893,ACE,3.333333,--,--,...,277.082115,0.9893,GALAXY,3.333333,2,0.0,0.9893,--|--|--,3,"flag disagreement, take mean. doublet secures z."
529,35185772185257520,-1,0.33.0.dev3890,0,GALAXY,0.9893,DRP,3.333333,--,--,...,277.082115,0.9893,GALAXY,3.333333,2,0.0,0.9893,--|--|--,3,"flag disagreement, take mean. doublet secures z."
579,35185772185257520,-1,0.33.0.dev3890,0,GALAXY,0.9893,TWL,3.333333,--,--,...,277.082115,0.9893,GALAXY,3.333333,2,0.0,0.9893,--|--|--,3,"flag disagreement, take mean. doublet secures z."


conflict ID =  38


In [118]:
#copy this text to a new cell to display the conflict
conflict_id=39
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
487,35185772185259110,GALAXY,1.499,ACE,3.0,--,--,--,--,1.499,GALAXY,1.333333,3,1.499,0.0,--|--| Model fits sky residuals,3,none
537,35185772185259110,GALAXY,1.499,DRP,0.0,--,--,--,--,1.499,GALAXY,1.333333,3,1.499,0.0,--|--| Model fits sky residuals,3,none
587,35185772185259110,GALAXY,1.499,TWL,1.0,--,--,--,Model fits sky residuals,1.499,GALAXY,1.333333,3,1.499,0.0,--|--| Model fits sky residuals,3,none


In [119]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. line in skyline region, no continuum. not secure. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
487,35185772185259110,-1,0.33.0.dev3890,0,GALAXY,1.499,ACE,1.333333,--,--,...,3.868085,1.499,GALAXY,1.333333,3,0.0,1.499,--|--| Model fits sky residuals,3,"flag disagreement, take mean. line in skyline ..."
537,35185772185259110,-1,0.33.0.dev3890,0,GALAXY,1.499,DRP,1.333333,--,--,...,3.868085,1.499,GALAXY,1.333333,3,0.0,1.499,--|--| Model fits sky residuals,3,"flag disagreement, take mean. line in skyline ..."
587,35185772185259110,-1,0.33.0.dev3890,0,GALAXY,1.499,TWL,1.333333,--,--,...,3.868085,1.499,GALAXY,1.333333,3,0.0,1.499,--|--| Model fits sky residuals,3,"flag disagreement, take mean. line in skyline ..."


conflict ID =  39


In [122]:
#copy this text to a new cell to display the conflict
conflict_id=40
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
488,35185772185259307,STAR,0.0002,ACE,4.0,--,--,--,--,0.0002,STAR,3.333333,2,0.0002,0.0,--|--|--,3,none
538,35185772185259307,STAR,0.0002,DRP,2.0,--,--,--,--,0.0002,STAR,3.333333,2,0.0002,0.0,--|--|--,3,none
588,35185772185259307,STAR,0.0002,TWL,4.0,--,--,--,--,0.0002,STAR,3.333333,2,0.0002,0.0,--|--|--,3,none


In [123]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
488,35185772185259307,-1,0.33.0.dev3890,0,STAR,0.0002,ACE,3.333333,--,--,...,77.995619,0.0002,STAR,3.333333,2,0.0,0.0002,--|--|--,3,"flag disagreement, take mean. z is secure."
538,35185772185259307,-1,0.33.0.dev3890,0,STAR,0.0002,DRP,3.333333,--,--,...,77.995619,0.0002,STAR,3.333333,2,0.0,0.0002,--|--|--,3,"flag disagreement, take mean. z is secure."
588,35185772185259307,-1,0.33.0.dev3890,0,STAR,0.0002,TWL,3.333333,--,--,...,77.995619,0.0002,STAR,3.333333,2,0.0,0.0002,--|--|--,3,"flag disagreement, take mean. z is secure."


conflict ID =  40


In [126]:
#copy this text to a new cell to display the conflict
conflict_id=41
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
493,35185772189451229,GALAXY,1.2595,ACE,4.0,--,--,--,--,1.2595,GALAXY,3.0,2,1.2595,0.0,--|--| Possible Broad [OII] in skyline regions,3,none
543,35185772189451229,GALAXY,1.2595,DRP,2.0,--,--,--,--,1.2595,GALAXY,3.0,2,1.2595,0.0,--|--| Possible Broad [OII] in skyline regions,3,none
593,35185772189451229,GALAXY,1.2595,TWL,3.0,--,--,--,Possible Broad [OII] in skyline regions,1.2595,GALAXY,3.0,2,1.2595,0.0,--|--| Possible Broad [OII] in skyline regions,3,none


In [127]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line. no secure continuum. z not secure. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
493,35185772189451229,-1,0.33.0.dev3890,0,GALAXY,1.2595,ACE,2.0,--,--,...,119.556172,1.2595,GALAXY,3.0,2,0.0,1.2595,--|--| Possible Broad [OII] in skyline regions,3,disagreement in flag. broadened but unresolved...
543,35185772189451229,-1,0.33.0.dev3890,0,GALAXY,1.2595,DRP,2.0,--,--,...,119.556172,1.2595,GALAXY,3.0,2,0.0,1.2595,--|--| Possible Broad [OII] in skyline regions,3,disagreement in flag. broadened but unresolved...
593,35185772189451229,-1,0.33.0.dev3890,0,GALAXY,1.2595,TWL,2.0,--,--,...,119.556172,1.2595,GALAXY,3.0,2,0.0,1.2595,--|--| Possible Broad [OII] in skyline regions,3,disagreement in flag. broadened but unresolved...


conflict ID =  41


In [130]:
#copy this text to a new cell to display the conflict
conflict_id=42
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
450,35185778174722907,GALAXY,1.1967,ACE,4.0,--,--,--,--,1.1967,GALAXY,3.0,2,1.1967,0.0,--|--|Broad [OII] emission,3,none
499,35185778174722907,GALAXY,1.1967,DRP,2.0,--,--,--,--,1.1967,GALAXY,3.0,2,1.1967,0.0,--|--|Broad [OII] emission,3,none
549,35185778174722907,GALAXY,1.1967,TWL,3.0,--,--,--,Broad [OII] emission,1.1967,GALAXY,3.0,2,1.1967,0.0,--|--|Broad [OII] emission,3,none


In [131]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. broadened but unresolved line. continuum is consistent with OII. z secure. poor sky subtraction'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
450,35185778174722907,-1,0.33.0.dev3890,0,GALAXY,1.1967,ACE,3.0,--,--,...,195.904517,1.1967,GALAXY,3.0,2,0.0,1.1967,--|--|Broad [OII] emission,3,"flag disagreement, take mean. broadened but un..."
499,35185778174722907,-1,0.33.0.dev3890,0,GALAXY,1.1967,DRP,3.0,--,--,...,195.904517,1.1967,GALAXY,3.0,2,0.0,1.1967,--|--|Broad [OII] emission,3,"flag disagreement, take mean. broadened but un..."
549,35185778174722907,-1,0.33.0.dev3890,0,GALAXY,1.1967,TWL,3.0,--,--,...,195.904517,1.1967,GALAXY,3.0,2,0.0,1.1967,--|--|Broad [OII] emission,3,"flag disagreement, take mean. broadened but un..."


conflict ID =  42


In [134]:
#copy this text to a new cell to display the conflict
conflict_id=43
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
457,35185778174727087,GALAXY,1.2653,ACE,3.0,--,--,--,--,1.2653,GALAXY,3.0,2,1.2653,0.0,--|--|--,3,none
506,35185778174727087,GALAXY,1.2653,DRP,2.0,--,--,--,--,1.2653,GALAXY,3.0,2,1.2653,0.0,--|--|--,3,none
556,35185778174727087,GALAXY,1.2653,TWL,4.0,--,--,--,--,1.2653,GALAXY,3.0,2,1.2653,0.0,--|--|--,3,none


In [136]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
457,35185778174727087,-1,0.33.0.dev3890,0,GALAXY,1.2653,ACE,3.0,--,--,...,109.398628,1.2653,GALAXY,3.0,2,0.0,1.2653,--|--|--,3,"flag disagreement, take mean. doublet secures z."
506,35185778174727087,-1,0.33.0.dev3890,0,GALAXY,1.2653,DRP,3.0,--,--,...,109.398628,1.2653,GALAXY,3.0,2,0.0,1.2653,--|--|--,3,"flag disagreement, take mean. doublet secures z."
556,35185778174727087,-1,0.33.0.dev3890,0,GALAXY,1.2653,TWL,3.0,--,--,...,109.398628,1.2653,GALAXY,3.0,2,0.0,1.2653,--|--|--,3,"flag disagreement, take mean. doublet secures z."


conflict ID =  43


In [139]:
#copy this text to a new cell to display the conflict
conflict_id=44
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
458,35185778174727095,GALAXY,1.3029,ACE,3.0,--,--,--,--,1.3029,GALAXY,3.0,2,1.3029,0.0,--|--|--,3,none
507,35185778174727095,GALAXY,1.3029,DRP,2.0,--,--,--,--,1.3029,GALAXY,3.0,2,1.3029,0.0,--|--|--,3,none
557,35185778174727095,GALAXY,1.3029,TWL,4.0,--,--,--,--,1.3029,GALAXY,3.0,2,1.3029,0.0,--|--|--,3,none


In [140]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
458,35185778174727095,-1,0.33.0.dev3890,0,GALAXY,1.3029,ACE,3.0,--,--,...,157.049318,1.3029,GALAXY,3.0,2,0.0,1.3029,--|--|--,3,"flag disagreement, take mean. doublet secures z."
507,35185778174727095,-1,0.33.0.dev3890,0,GALAXY,1.3029,DRP,3.0,--,--,...,157.049318,1.3029,GALAXY,3.0,2,0.0,1.3029,--|--|--,3,"flag disagreement, take mean. doublet secures z."
557,35185778174727095,-1,0.33.0.dev3890,0,GALAXY,1.3029,TWL,3.0,--,--,...,157.049318,1.3029,GALAXY,3.0,2,0.0,1.3029,--|--|--,3,"flag disagreement, take mean. doublet secures z."


conflict ID =  44


In [143]:
#copy this text to a new cell to display the conflict
conflict_id= 45
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
605,35185778178918499,GALAXY,1.5443,TWL,4.0,RC,0.0003,STAR,A point source,0.0003,STAR,3.0,3,1.5443,1.5443,A point source|--| White dwarf,3,none
655,35185778178918499,GALAXY,1.5443,SEM,1.0,R,--,--,--,1.5443,GALAXY,3.0,3,1.5443,1.5443,A point source|--| White dwarf,3,none
705,35185778178918499,GALAXY,1.5443,ACE,4.0,RC,0.000,STAR,White dwarf,0.0000,STAR,3.0,3,1.5443,1.5443,A point source|--| White dwarf,3,none


In [144]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.0003
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'STAR'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. 2/3 inspectors suggest STAR. consistent with magnetic WD. take VI with flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
605,35185778178918499,-1,0.33.0.dev3890,0,GALAXY,1.5443,TWL,3.0,RC,0.0003,...,2.098335,0.0003,STAR,3.0,3,1.5443,1.5443,A point source|--| White dwarf,3,disagreement in all. 2/3 inspectors suggest ST...
655,35185778178918499,-1,0.33.0.dev3890,0,GALAXY,1.5443,SEM,3.0,R,--,...,2.098335,0.0003,STAR,3.0,3,1.5443,1.5443,A point source|--| White dwarf,3,disagreement in all. 2/3 inspectors suggest ST...
705,35185778178918499,-1,0.33.0.dev3890,0,GALAXY,1.5443,ACE,3.0,RC,0.000,...,2.098335,0.0003,STAR,3.0,3,1.5443,1.5443,A point source|--| White dwarf,3,disagreement in all. 2/3 inspectors suggest ST...


conflict ID =  45


In [147]:
#copy this text to a new cell to display the conflict
conflict_id=46
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
618,35185778183111392,GALAXY,0.6759,TWL,2.0,--,--,--,Not sure the best fit model gives a right red...,0.6759,GALAXY,2.0,2,0.6759,0.0,Not sure the best fit model gives a right red...,3,none
668,35185778183111392,GALAXY,0.6759,SEM,3.0,--,--,--,balmer lines and OII and HeII,0.6759,GALAXY,2.0,2,0.6759,0.0,Not sure the best fit model gives a right red...,3,none
718,35185778183111392,GALAXY,0.6759,ACE,1.0,--,--,--,No signal.,0.6759,GALAXY,2.0,2,0.6759,0.0,Not sure the best fit model gives a right red...,3,none


In [148]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
618,35185778183111392,-1,0.33.0.dev3890,0,GALAXY,0.6759,TWL,2.0,--,--,...,27.584781,0.6759,GALAXY,2.0,2,0.0,0.6759,Not sure the best fit model gives a right red...,3,"flag disagreement, take mean. z not secure."
668,35185778183111392,-1,0.33.0.dev3890,0,GALAXY,0.6759,SEM,2.0,--,--,...,27.584781,0.6759,GALAXY,2.0,2,0.0,0.6759,Not sure the best fit model gives a right red...,3,"flag disagreement, take mean. z not secure."
718,35185778183111392,-1,0.33.0.dev3890,0,GALAXY,0.6759,ACE,2.0,--,--,...,27.584781,0.6759,GALAXY,2.0,2,0.0,0.6759,Not sure the best fit model gives a right red...,3,"flag disagreement, take mean. z not secure."


conflict ID =  46


In [151]:
#copy this text to a new cell to display the conflict
conflict_id=47
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
639,35185778183115498,GALAXY,0.7653,TWL,4.0,S,1.0899,--,Calibration feature,1.0899,GALAXY,2.666667,4,0.7653,0.183991,Calibration feature| gap in spectrum|Bad conti...,3,none
689,35185778183115498,GALAXY,0.7653,SEM,0.0,--,--,--,gap in spectrum,0.7653,GALAXY,2.666667,4,0.7653,0.183991,Calibration feature| gap in spectrum|Bad conti...,3,none
739,35185778183115498,GALAXY,0.7653,ACE,4.0,S,1.0901,GALAXY,Bad continuum subtraction 6800-7600A. Clear OI...,1.0901,GALAXY,2.666667,4,0.7653,0.183991,Calibration feature| gap in spectrum|Bad conti...,3,none


In [152]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.090
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/3 inspectors give VI z based on doublet missed by RR. take with their mean flag of 4. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
639,35185778183115498,-1,0.33.0.dev3890,0,GALAXY,0.7653,TWL,4.0,S,1.0899,...,123.97624,1.09,GALAXY,2.666667,4,0.183991,0.7653,Calibration feature| gap in spectrum|Bad conti...,3,disagreement in z and flag. 2/3 inspectors giv...
689,35185778183115498,-1,0.33.0.dev3890,0,GALAXY,0.7653,SEM,4.0,--,--,...,123.97624,1.09,GALAXY,2.666667,4,0.183991,0.7653,Calibration feature| gap in spectrum|Bad conti...,3,disagreement in z and flag. 2/3 inspectors giv...
739,35185778183115498,-1,0.33.0.dev3890,0,GALAXY,0.7653,ACE,4.0,S,1.0901,...,123.97624,1.09,GALAXY,2.666667,4,0.183991,0.7653,Calibration feature| gap in spectrum|Bad conti...,3,disagreement in z and flag. 2/3 inspectors giv...


conflict ID =  47


In [155]:
#copy this text to a new cell to display the conflict
conflict_id=48
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
641,35185778187305121,GALAXY,1.2621,TWL,4.0,--,--,--,Broad [OII] emission,1.2621,GALAXY,4.0,0,1.2621,0.0,Broad [OII] emission| OII and balmer in absor...,3,none
691,35185778187305121,GALAXY,1.2621,SEM,4.0,--,--,--,OII and balmer in absorption,1.2621,GALAXY,4.0,0,1.2621,0.0,Broad [OII] emission| OII and balmer in absor...,3,none
741,35185778187305121,GALAXY,1.2621,ACE,4.0,--,1.2621,QSO,Broad MgII and CIII] emission lines.,1.2621,QSO,4.0,0,1.2621,0.0,Broad [OII] emission| OII and balmer in absor...,3,none


In [156]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype. possible missed broad MgII emission. set to QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
641,35185778187305121,-1,0.33.0.dev3890,0,GALAXY,1.2621,TWL,4.0,--,--,...,281.366725,1.2621,QSO,4.0,0,0.0,1.2621,Broad [OII] emission| OII and balmer in absor...,3,disagreement in spectype. possible missed broa...
691,35185778187305121,-1,0.33.0.dev3890,0,GALAXY,1.2621,SEM,4.0,--,--,...,281.366725,1.2621,QSO,4.0,0,0.0,1.2621,Broad [OII] emission| OII and balmer in absor...,3,disagreement in spectype. possible missed broa...
741,35185778187305121,-1,0.33.0.dev3890,0,GALAXY,1.2621,ACE,4.0,--,1.2621,...,281.366725,1.2621,QSO,4.0,0,0.0,1.2621,Broad [OII] emission| OII and balmer in absor...,3,disagreement in spectype. possible missed broa...


conflict ID =  48


### and so on...

We should now recompute the conflicts, and not find any.

In [159]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [160]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

Let's check the number of conflicts again.

In [161]:
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = VI.find_conflicts(vi_gp)

In [162]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [163]:
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [164]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [165]:
print(VI_dir+save_filename)

/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/truth_table_ELGs_sets26_30.txt


In [166]:
#vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+'merged_VI.txt')
vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments', 'merger comment','DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+save_filename)

Check that merged file reads in OK - check comments

In [167]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [168]:
merged_file.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments',
       'merger comment', 'DELTACHI2', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z'],
      dtype='object')

In [169]:
merged_file

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35185772147511978,1.3313,GALAXY,1.3313,GALAXY,1.333333,3,--| Low S/N|No clear match to features,none,4.145577,1563,0.841889,0.836042,1.296284,0.431983,0.428982,0.665138
1,35185772147512223,0.8267,GALAXY,0.8267,GALAXY,4.000000,3,--|--|--,none,1112.861230,1841,0.335037,0.475023,0.912325,0.260517,0.369368,0.709405
2,35185772147512575,1.6979,GALAXY,1.6979,GALAXY,1.000000,3,--| Some continuum; no features|--,"disagreement in flag and spectype. low SNR, ha...",0.358240,1833,0.382729,0.577815,1.517927,0.155166,0.234258,0.615399
3,35185772151701749,1.6163,GALAXY,1.6163,GALAXY,2.500000,3,--|Calibration features around 5800 A; possibl...,disagreement in flag. broadened but just about...,56.813565,1270,0.252087,0.285118,0.272102,0.196316,0.222039,0.211903
4,35185772151701829,0.4462,GALAXY,0.4462,GALAXY,0.666667,3,--| Calibration features around 5800A| Poor co...,none,66.079907,1403,0.460372,0.537241,0.728247,0.320315,0.373799,0.506696
5,35185772151702242,0.9726,GALAXY,0.9726,GALAXY,3.666667,3,--| Calibration features around 6000 A| Poor c...,none,61.165908,1271,0.621843,0.893893,2.119749,0.201883,0.290205,0.688182
6,35185772151702443,1.0765,GALAXY,1.0765,GALAXY,4.000000,3,--| Broad [OII] emission; Calibration features...,none,501.384285,1430,1.247653,1.719018,3.747288,0.658575,0.907385,1.978010
7,35185772151702723,1.6776,QSO,1.6776,QSO,1.000000,3,--| Calibration features between 6000 - 7000A|...,none,13.514251,1288,0.386234,0.422026,0.537866,0.241947,0.264368,0.336933
8,35185772151702819,0.5701,GALAXY,0.5701,GALAXY,1.000000,3,--| Bad calibration| Based mainly on SiIV,none,6.296079,1278,0.417958,0.788193,3.091209,0.228929,0.431718,1.693153
9,35185772151703841,1.2142,GALAXY,1.2142,GALAXY,3.000000,3,--| Broad [OII] ; calibration features around ...,"disagreement in flag, but OII secures z. calib...",71.271587,1416,0.480862,0.409178,0.839671,0.267762,0.227845,0.467560


In [170]:
merged_file.loc[9]['merger comment']

'disagreement in flag, but OII secures z. calibration issues. poor sky subtraction.'

In [171]:
merged_file.loc[9]['all VI comments']

'--| Broad [OII] ; calibration features around 5800 A|  Poor continuum subtraction at 5800-6800A.'

In [172]:
VI_dir

'/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/'